In [1]:
import pandas as pd 
import numpy as np 

In [10]:
df = pd.read_excel("Case - Analytics (1).xlsx", engine = 'openpyxl')
print(df.info())
df = df.dropna()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3070 entries, 0 to 3069
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     2636 non-null   datetime64[ns]
 1   user_id  2636 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 48.1 KB
None


,date,user_id
0,2020-03-04,4.0
1,2020-03-04,22.0
2,2020-03-04,26.0
3,2020-03-04,36.0
4,2020-03-04,38.0
...,...,...
2631,2020-01-02,10.0
2632,2020-01-02,11.0
2633,2020-01-02,12.0
2634,2020-01-02,13.0


# 1. Activity

In [22]:
df['week'] = df['date'].dt.weekofyear
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek

In [57]:
df

,date,user_id,week,month,day,day_of_week
0,2020-03-04,4.0,10,3,4,2
1,2020-03-04,22.0,10,3,4,2
2,2020-03-04,26.0,10,3,4,2
3,2020-03-04,36.0,10,3,4,2
4,2020-03-04,38.0,10,3,4,2
...,...,...,...,...,...,...
2631,2020-01-02,10.0,1,1,2,3
2632,2020-01-02,11.0,1,1,2,3
2633,2020-01-02,12.0,1,1,2,3
2634,2020-01-02,13.0,1,1,2,3


## Activity by day: On average, 14 unique users used this product per day

In [72]:
day_unique = df.drop_duplicates(subset = ['user_id', 'month', 'week', 'day'])
day_unique = day_unique.reset_index(drop = True)
daily_activity = day_unique.groupby(['month', 'week', 'day'])['user_id'].agg(['count']).reset_index()
daily_activity.describe()

,month,week,day,count
count,62.000000,62.000000,62.000000,62.000000
mean,1.580645,5.370968,14.693548,42.516129
std,0.615334,2.631673,8.845836,25.444728
min,1.000000,1.000000,1.000000,5.000000
25%,1.000000,3.000000,7.000000,23.000000
50%,2.000000,5.000000,14.500000,41.000000
75%,2.000000,7.750000,22.000000,60.500000
max,3.000000,10.000000,31.000000,97.000000


## Activity by day by user: There are 255 unique users of this product, about 120 used this product between 8 and 41 days, while the other 120 used this product between 1 and 8 days. 

In [95]:
total_user_activity = day_unique.groupby(['user_id'])['day_of_week'].agg(['count']).reset_index()
total_user_activity.describe()

,user_id,count
count,255.000000,255.000000
mean,133.133333,10.337255
std,76.207225,8.016468
min,1.000000,1.000000
25%,67.500000,4.000000
50%,134.000000,8.000000
75%,198.500000,14.000000
max,263.000000,41.000000


## Activity by week - The number of unique users increases every week. NOTE: Week 10 is not a full week: Mon, Tue, Wed only

In [37]:
week_unique = df.drop_duplicates(subset = ['user_id', 'week'])
week_unique = week_unique.reset_index(drop = True)
week_activity = week_unique.groupby(['week'])['user_id'].agg(['count']).reset_index()
week_activity

,week,count
0,1,14
1,2,29
2,3,45
3,4,65
4,5,86
5,6,99
6,7,121
7,8,147
8,9,180
9,10,148


## Activity by week by user: About half the users used this product between 3 and 9 weeks, while the other half used it between 1 and 3 weeks. 

In [96]:
total_weekly_user_activity = week_unique.groupby(['user_id'])['day_of_week'].agg(['count']).reset_index()
total_weekly_user_activity.describe()

,user_id,count
count,255.000000,255.000000
mean,133.133333,3.662745
std,76.207225,1.999012
min,1.000000,1.000000
25%,67.500000,2.000000
50%,134.000000,3.000000
75%,198.500000,5.000000
max,263.000000,10.000000


In [99]:
total_weekly_user_activity

,user_id,count
0,1.0,7
1,2.0,5
2,3.0,5
3,4.0,10
4,5.0,3
...,...,...
250,259.0,1
251,260.0,1
252,261.0,1
253,262.0,2


## Activity by Month: not useful as of now because one can only compare Jan and Feb since March is only 4 days, but based on those 4 days it already has a lot of activity and so we would still expect activity to increase NOTE: March only has 4 days compared to January and February which are full months

In [48]:
month_unique = df.drop_duplicates(subset = ['user_id', 'month'])
month_unique = month_unique.reset_index(drop = True)
month_activity = month_unique.groupby(['month'])['user_id'].agg(['count']).reset_index()
month_activity

,month,count
0,1,86
1,2,232
2,3,160


## Activity by month by user: 25% of users used the product in all 3 months (with March only having 3 working days), so we expect this to increase. 75% of users used the product for at least 2 months. With only 2 months and a small part of a 3rd month, this is something to track for the future. 

In [98]:
total_monthly_user_activity = month_unique.groupby(['user_id'])['day_of_week'].agg(['count']).reset_index()
total_monthly_user_activity.describe()

,user_id,count
count,255.000000,255.000000
mean,133.133333,1.874510
std,76.207225,0.595205
min,1.000000,1.000000
25%,67.500000,2.000000
50%,134.000000,2.000000
75%,198.500000,2.000000
max,263.000000,3.000000


# 2. Growth

## Growth by day- given the sample sizes I do not focus on takeaways from daily growth, one after another, but rather: What does the average day's growth look like? The average day over the 9 weeks has about 6% growth. Because the average percentage growth is larger than the median percentage growth, it means some days have a lot of growth. Additionally, half of the day's have between 0% and 150% growth, while half of the days have between 37% decrease in growth and no growth.

In [69]:
daily_activity['previous_day_cnts'] = daily_activity['count'].shift(periods = 1, axis = 'rows')
daily_activity['growth_counts'] = daily_activity['count'] - daily_activity['previous_day_cnts']
daily_activity['growth_pct'] = (daily_activity['growth_counts'] / daily_activity['previous_day_cnts']) * 100
daily_activity[['growth_counts', 'growth_pct']].describe()

,growth_counts,growth_pct
count,61.000000,61.000000
mean,1.114754,5.949567
std,7.552259,26.618142
min,-14.000000,-37.500000
0%,-14.000000,-37.500000
25%,-3.000000,-7.407407
50%,0.000000,0.000000
max,27.000000,150.000000


## Growth by week- The user base grows every week, and half the weeks experience 27% increase in growth or more, or 20+ users. There is weekly growth anywhere between 15 and 107 percent, and is growing steadily each week around 20% growth each week for the last 3 weeks. Note:The 10th week is only 3 days and so is a partial week- therefore I am not including it.

In [70]:
week_activity['previous_week_cnts'] = week_activity['count'].shift(periods = 1, axis = 'rows')
week_activity['growth_counts'] = week_activity['count'] - week_activity['previous_week_cnts']
week_activity['growth_pct'] = (week_activity['growth_counts'] / week_activity['previous_week_cnts']) * 100
week_activity[:9]

,week,count,previous_week_cnts,growth_counts,growth_pct
0,1,14,NaN,NaN,NaN
1,2,29,14.0,15.0,107.142857
2,3,45,29.0,16.0,55.172414
3,4,65,45.0,20.0,44.444444
4,5,86,65.0,21.0,32.307692
5,6,99,86.0,13.0,15.116279
6,7,121,99.0,22.0,22.222222
7,8,147,121.0,26.0,21.487603
8,9,180,147.0,33.0,22.448980


In [71]:
week_activity[:9][['growth_counts', 'growth_pct']].describe()


,growth_counts,growth_pct
count,8.000000,8.000000
mean,20.750000,40.042811
std,6.497252,30.244885
min,13.000000,15.116279
25%,15.750000,22.038567
50%,20.500000,27.378336
75%,23.000000,47.126437
max,33.000000,107.142857


## Growth by month- not useful as of now because one can only compare Jan and Feb since March is only 4 days, but based on those 4 days it already has a lot of growth and so we would still expect growth to increase

In [56]:
#month_activity = month_activity.sort_values(by = ['month'], ascending = False)
month_activity['previous_month_cnts'] = month_activity['count'].shift(periods = 1, axis = 'rows')
month_activity['growth_counts'] = month_activity['count'] - month_activity['previous_month_cnts']
month_activity['growth_pct'] = (month_activity['growth_counts'] / month_activity['previous_month_cnts']) * 100
month_activity

,month,count,previous_month_cnts,growth_counts,growth_pct
0,1,86,NaN,NaN,NaN
1,2,232,86.0,146.0,169.767442
2,3,160,232.0,-72.0,-31.034483


# 3. Profitability: After 9 weeks the product is not yet profitable-but will likely become profitable early next week, as only 67 more users are required to reach profitability, and 66 to reach break-even

In [76]:
week_activity['profit_per_week'] = week_activity['count'] * 2
week_activity

,week,count,previous_week_cnts,growth_counts,growth_pct,profit_per_week
0,1,14,NaN,NaN,NaN,28
1,2,29,14.0,15.0,107.142857,58
2,3,45,29.0,16.0,55.172414,90
3,4,65,45.0,20.0,44.444444,130
4,5,86,65.0,21.0,32.307692,172
5,6,99,86.0,13.0,15.116279,198
6,7,121,99.0,22.0,22.222222,242
7,8,147,121.0,26.0,21.487603,294
8,9,180,147.0,33.0,22.448980,360
9,10,148,180.0,-32.0,-17.777778,296


In [78]:
week_activity['profit_per_week'].sum()

1868

## Number of users until profitability

In [80]:
(2000 - week_activity['profit_per_week'].sum()) / 2

66.0

## When thinking about interesting metrics I assume that one wants to maximize growth and retention for product market fit. There is presumably some limit to the total pool of people interested in this product. If lots of people want to try the product, that is great for growth, but if they all drop off or most drop off, then the company would be continuously looking for new people to try the product, and then retain them.  Maximizing growth and retention requires 1. gaining users and 2. retaining them. Therefore I will examine individual user patterns, and individual user patterns over time. I assume there could be differences between someone who began with the product in Jan because by March, presumably improvements have been made, and features added that might retain more people, that could have dropped off in Jan. Retention: Someone who uses the product (we do not know for how long).

With the current amount of data, we will focus on week-over-week retention, because profitability is calculated based on Avg Weekly Users. 
- How has user retention changed over time? ie. Are users retaining better in late Feb/March better than in early Jan? User retention cohort
- Is there a strong point of drop-off (ie. after 3 weeks, many users stop completely)
- How strong an indicator is daily use an indicator of weekly retention? Ie if most users use the product for 5 days in a row, are they more likely to keep using it than someone who uses it once a week? One would use different hooks to bring someone back daily rather than 1x a week

Differences in how people are retained and use the product impacts....

Marketing / Product
Additionally, it is important to think about how to people use the product to retain them. If most users who retain use the product every day, there are likely different hooks and nudges needed both when marketing the product and when interacting with users (ie if it is an app you may send daily push notifications of some kind). However, if most people use the product once a week, but retain, then having daily notifications (if it was an app), might be annoying and discourage the user. 

Data Science
You will create different metrics and analyses based on retained user patterns. 

Engineering
If people use a product 1x a week versus 5x a week there is a different amount of load required- similarly if people are all using the product at the same time versus at different times.

## Week 1 User retention

In [127]:
week1_users = week_unique[week_unique['week'] == 1]
print(week1_users.user_id.nunique(), "initial user count")
week2_users = week_unique[week_unique['week'] == 2]
week3_users = week_unique[week_unique['week'] == 3]
week4_users = week_unique[week_unique['week'] == 4]
week5_users = week_unique[week_unique['week'] == 5]
week6_users = week_unique[week_unique['week'] == 6]
week7_users = week_unique[week_unique['week'] == 7]
week8_users = week_unique[week_unique['week'] == 8]
week9_users = week_unique[week_unique['week'] == 9]
week10_users = week_unique[week_unique['week'] == 10]

initial_count = week1_users.user_id.nunique()
week1_pct_list = list()

all_weeks = [week1_users, week2_users, week3_users, week4_users, week5_users, week6_users, week7_users, week8_users, week9_users, week10_users]
for week_number in range(1, len(all_weeks)):
    print(len(all_weeks[week_number][all_weeks[week_number]['user_id'].isin(week1_users.user_id)]), "count retained after", week_number, "Weeks")
    retained_count = len(all_weeks[week_number][all_weeks[week_number]['user_id'].isin(week1_users.user_id)])
    print(round((retained_count / initial_count) * 100,2), "pct")
    week1_pct_list.append((round((retained_count / initial_count) * 100,2), "week number", week_number))
week1_pct_list

14 initial user count
14 count retained after 1 Weeks
100.0 pct
13 count retained after 2 Weeks
92.86 pct
12 count retained after 3 Weeks
85.71 pct
12 count retained after 4 Weeks
85.71 pct
10 count retained after 5 Weeks
71.43 pct
8 count retained after 6 Weeks
57.14 pct
5 count retained after 7 Weeks
35.71 pct
4 count retained after 8 Weeks
28.57 pct
4 count retained after 9 Weeks
28.57 pct


[(100.0, 'week number', 1),
 (92.86, 'week number', 2),
 (85.71, 'week number', 3),
 (85.71, 'week number', 4),
 (71.43, 'week number', 5),
 (57.14, 'week number', 6),
 (35.71, 'week number', 7),
 (28.57, 'week number', 8),
 (28.57, 'week number', 9)]

## Week 4 User retention over time

In [129]:
print(week4_users.user_id.nunique(), "initial user count")

initial_count = week4_users.user_id.nunique()
week4_pct_list = list()

all_weeks = [week4_users, week5_users, week6_users, week7_users, week8_users, week9_users, week10_users]
for week_number in range(1, len(all_weeks)):
    print(len(all_weeks[week_number][all_weeks[week_number]['user_id'].isin(week4_users.user_id)]), "count retained after", week_number, "Weeks")
    retained_count = len(all_weeks[week_number][all_weeks[week_number]['user_id'].isin(week4_users.user_id)])
    print(round((retained_count / initial_count) * 100,2), "pct")
    week4_pct_list.append((round((retained_count / initial_count) * 100,2), "week number", week_number))
week4_pct_list

65 initial user count
63 count retained after 1 Weeks
96.92 pct
53 count retained after 2 Weeks
81.54 pct
40 count retained after 3 Weeks
61.54 pct
28 count retained after 4 Weeks
43.08 pct
23 count retained after 5 Weeks
35.38 pct
16 count retained after 6 Weeks
24.62 pct


[(96.92, 'week number', 1),
 (81.54, 'week number', 2),
 (61.54, 'week number', 3),
 (43.08, 'week number', 4),
 (35.38, 'week number', 5),
 (24.62, 'week number', 6)]

## Week 6 User retention

In [132]:
print(week6_users.user_id.nunique(), "initial user count")

initial_count = week6_users.user_id.nunique()
week6_pct_list = list()

all_weeks = [week6_users, week7_users, week8_users, week9_users, week10_users]
for week_number in range(1, len(all_weeks)):
    print(len(all_weeks[week_number][all_weeks[week_number]['user_id'].isin(week6_users.user_id)]), "count retained after", week_number, "Weeks")
    retained_count = len(all_weeks[week_number][all_weeks[week_number]['user_id'].isin(week6_users.user_id)])
    print(round((retained_count / initial_count) * 100,2), "pct")
    week6_pct_list.append((round((retained_count / initial_count) * 100,2), "week number", week_number))
week6_pct_list

99 initial user count
84 count retained after 1 Weeks
84.85 pct
70 count retained after 2 Weeks
70.71 pct
54 count retained after 3 Weeks
54.55 pct
45 count retained after 4 Weeks
45.45 pct


[(84.85, 'week number', 1),
 (70.71, 'week number', 2),
 (54.55, 'week number', 3),
 (45.45, 'week number', 4)]

## Retention by Cohort Analysis

In [137]:
retention_by_cohort = pd.DataFrame({'week1_rentention' : week1_pct_list})
rt4 = pd.DataFrame({'week4_retention' : week4_pct_list})
retention_by_cohort = retention_by_cohort.merge(rt4, left_index = True, right_index = True, how = 'outer')
rt6 = pd.DataFrame({'week4_retention' : week6_pct_list})
retention_by_cohort = retention_by_cohort.merge(rt6, left_index = True, right_index = True, how = 'outer')
retention_by_cohort

,week1_rentention,week4_retention_x,week4_retention_y
0,"(100.0, week number, 1)","(96.92, week number, 1)","(84.85, week number, 1)"
1,"(92.86, week number, 2)","(81.54, week number, 2)","(70.71, week number, 2)"
2,"(85.71, week number, 3)","(61.54, week number, 3)","(54.55, week number, 3)"
3,"(85.71, week number, 4)","(43.08, week number, 4)","(45.45, week number, 4)"
4,"(71.43, week number, 5)","(35.38, week number, 5)",NaN
5,"(57.14, week number, 6)","(24.62, week number, 6)",NaN
6,"(35.71, week number, 7)",NaN,NaN
7,"(28.57, week number, 8)",NaN,NaN
8,"(28.57, week number, 9)",NaN,NaN
